In [14]:
from data_selector import DataSelector

In [15]:
ds = DataSelector("data.csv")

In [16]:
a = ds.select_data(ratio={"Cornell": 1})

Filtering...
Calculating counts...
Grabbing data...


65835
